In [ ]:
from datasets import load_dataset, Audio

SAMPLING_RATE=24_000
# Load the LJ Speech dataset
dataset = load_dataset("MikhailT/lj-speech")
dataset = dataset.cast_column("audio", Audio(sampling_rate=SAMPLING_RATE))
dataset = dataset.with_format("torch")
len(dataset["full"])

In [ ]:
from data_pipeline.utils.codec import MimiCodec

mimi_model = MimiCodec()

In [ ]:
first_items = dataset["full"][0:16]
audios = [row["array"] for row in first_items["audio"]]
wavs = mimi_model.encode_batch(audios)

In [ ]:
[l.shape for l in wavs]

In [ ]:
from IPython import display

out_pcm = mimi_model.decode(wavs[15])

# Convert to CPU and get numpy array for playback
audio_data = out_pcm.numpy()

# Create and display audio widget
# Note: sample_rate=24000 matches your original save command
display.display(display.Audio(audio_data, rate=24000, autoplay=False))

In [ ]:
def batch_wav_encoder(batch):
    audios = [audio["array"] for audio in batch["audio"]]
    return {
        "codes": mimi_model.encode_batch(audios)
    }

dataset = dataset.map(batch_wav_encoder, batched=True, batch_size=24, remove_columns=["audio"])

In [ ]:
dataset.column_names

In [ ]:
dataset = dataset.rename_column(original_column_name="normalized_text", new_column_name="text_normalized")
dataset.save_to_disk("../../datasets/encoded_ljspeech")